In [1]:
import nest_asyncio

nest_asyncio.apply()
print("Event loop ready.")

Event loop ready.


In [6]:
from rasa.cli.scaffold import create_initial_project
import os

import rasa
from rasa.nlu.train import Interpreter
from rasa.nlu.training_data import load_data
from rasa.nlu.config import RasaNLUModelConfig
from rasa.nlu.model import Trainer
from rasa.nlu import config
import json

#Getting the config file
project = "test-project"
create_initial_project(project)

# move into project directory and show files
os.chdir(project)
print(os.listdir("."))

['actions.py', 'config.yml', 'credentials.yml', 'data', 'domain.yml', 'endpoints.yml', 'tests', '__init__.py', '__pycache__']


#### Creating the nlu.md file and Writing our own Intent

In [8]:
nlu_md = """
## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:restaurant_search
- i'm looking for a place to eat
- I want to grab lunch
- I am searching for a dinner spot
- i'm looking for a place in the north location of town
- show me  chinese cuisine restaurants
- show me a mexican cuisine place in the [centre](location)
- i am looking for an indian cuisine spot
- search for restaurants
- anywhere in the 
- anywhere near 
- I am looking for asian fusion  food
- I am looking a restaurant 

## intent:cancel_order
- Not hungry anymore
- Getting Your Food Is An Impossible Dream
- Thinking of eating something else
- Cheaper alternative available for lesser price.
- Bad review from friends
- come across with better deal 
- Delayed Delivery

## intent:Delivery status
- When will my order come
- why hasnt the order reached on time
- Cant wait any longer
- Tired of waiting
- why is there a delay
- will  i get my order today

## intent:thankyou
- thanks!
- thank you
- thx
- thanks very much
"""

#Saving the file
text_file = open("nlu.md", "w")
text_file.write(nlu_md)
text_file.close()

In [9]:
#loading the NLU training Example
training_data = load_data('nlu.md')

#Trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

#train The model
interpreter = trainer.train(training_data)

#Store it for future use
model_directory = trainer.persist('model',fixed_model_name = 'current')

C:\Users\deeks\Anaconda3\envs\tensorflow\lib\site-packages\rasa\utils\common.py:363: UserWarning: Entity entity 'location' has only 1 training examples! The minimum is 2, because of this the training may fail.
Epochs: 100%|█| 100/100 [00:31<00:00,  3.18it/s, t_loss=1.640, i_loss=0.015, entity_loss=0.009, i_acc=1.000, entity_f1=0.715]


#### Evaluating the NLU model

In [10]:
#A helper function for pretier output
def pprint(o):
    print(json.dumps(o,indent=3))

print(interpreter.parse('hello'))

{'intent': {'name': 'greet', 'confidence': 0.999999463558197}, 'entities': [], 'intent_ranking': [{'name': 'greet', 'confidence': 0.999999463558197}, {'name': 'cancel_order', 'confidence': 3.9446979371859925e-07}, {'name': 'Delivery status', 'confidence': 1.7007033648042125e-07}, {'name': 'thankyou', 'confidence': 3.2196725285871253e-09}, {'name': 'restaurant_search', 'confidence': 7.83079937699549e-12}], 'response_selector': {'default': {'response': {'name': None, 'confidence': 0.0}, 'ranking': [], 'full_retrieval_intent': 'thankyou/None'}}, 'text': 'hello'}


In [11]:
print(interpreter.parse("let's see some italian restaurants"))

{'intent': {'name': 'restaurant_search', 'confidence': 0.998565137386322}, 'entities': [], 'intent_ranking': [{'name': 'restaurant_search', 'confidence': 0.998565137386322}, {'name': 'Delivery status', 'confidence': 0.000838333333376795}, {'name': 'thankyou', 'confidence': 0.0005029566236771643}, {'name': 'cancel_order', 'confidence': 8.979772974271327e-05}, {'name': 'greet', 'confidence': 3.7276886359904893e-06}], 'response_selector': {'default': {'response': {'name': None, 'confidence': 0.0}, 'ranking': [], 'full_retrieval_intent': 'thankyou/None'}}, 'text': "let's see some italian restaurants"}


In [12]:
print(interpreter.parse("No longer hungry"))

{'intent': {'name': 'cancel_order', 'confidence': 0.8156499266624451}, 'entities': [], 'intent_ranking': [{'name': 'cancel_order', 'confidence': 0.8156499266624451}, {'name': 'Delivery status', 'confidence': 0.12390382587909698}, {'name': 'greet', 'confidence': 0.05915297940373421}, {'name': 'thankyou', 'confidence': 0.0009208808769471943}, {'name': 'restaurant_search', 'confidence': 0.000372402835637331}], 'response_selector': {'default': {'response': {'name': None, 'confidence': 0.0}, 'ranking': [], 'full_retrieval_intent': 'thankyou/None'}}, 'text': 'No longer hungry'}


In [13]:
print(interpreter.parse("How much longer do i have to wait"))

{'intent': {'name': 'Delivery status', 'confidence': 0.8699944615364075}, 'entities': [], 'intent_ranking': [{'name': 'Delivery status', 'confidence': 0.8699944615364075}, {'name': 'restaurant_search', 'confidence': 0.12810568511486053}, {'name': 'thankyou', 'confidence': 0.0013391440734267235}, {'name': 'cancel_order', 'confidence': 0.0002893274067901075}, {'name': 'greet', 'confidence': 0.00027141228201799095}], 'response_selector': {'default': {'response': {'name': None, 'confidence': 0.0}, 'ranking': [], 'full_retrieval_intent': 'thankyou/None'}}, 'text': 'How much longer do i have to wait'}
